In [1]:
import requests
import pandas as pd
import time  # To be polite to the API


In [5]:

def fetch_all_worldbank_documents(query, start_date=None, end_date=None, rows_per_page=100):
    """Fetch all documents matching a query across all countries and pages."""
    all_records = []
    offset = 0
    while True:
        base_url = "https://search.worldbank.org/api/v3/wds"
        params = {
            "format": "json",
            "qterm": query,
            "rows": rows_per_page,
            "os": offset,
            "fl": "display_title,docdt,docty,count,repnb,pdfurl"
        }
        if start_date:
            params["strdate"] = start_date
        if end_date:
            params["enddate"] = end_date
        response = requests.get(base_url, params=params)
        data = response.json()
        if not data or "documents" not in data or not data["documents"]:
            break
        for d in data["documents"].values():
            all_records.append({
                "Title": d.get("display_title"),
                "Date": d.get("docdt"),
                "Type": d.get("docty"),
                "Country": d.get("count"),
                "Report No": d.get("repnb"),
                "PDF Link": d.get("pdfurl")
            })
        offset += rows_per_page
    return pd.DataFrame(all_records)


In [6]:
# Example: all solar finance documents from all countries since 2021

df_all = fetch_all_worldbank_documents(query="solar finance", start_date="2021-01-01", rows_per_page=100)


In [7]:
# Save to CSV
df_all.to_csv("all_worldbank_solar_finance_docs.csv", index=False)

df_all.head()


,Title,Date,Type,Country,Report No,PDF Link
0,Burkina Faso - Solar Energy and Access Project,2021-05-21T04:00:00Z,Project Appraisal Document,Burkina Faso,PAD3364,http://documents.worldbank.org/curated/en/9881...
1,Pakistan - SOUTH ASIA- P159712- Sindh Solar En...,2021-03-01T05:00:00Z,Procurement Plan,Pakistan,STEP45794,http://documents.worldbank.org/curated/en/5946...
2,Pakistan - SOUTH ASIA- P159712- Sindh Solar En...,2021-09-27T04:00:00Z,Procurement Plan,Pakistan,STEP54823,http://documents.worldbank.org/curated/en/9406...
3,Pakistan - SOUTH ASIA- P159712- Sindh Solar En...,2021-10-26T04:00:00Z,Procurement Plan,Pakistan,STEP56129,http://documents.worldbank.org/curated/en/9461...
4,Pakistan - SOUTH ASIA- P159712- Sindh Solar En...,2021-06-09T04:00:00Z,Procurement Plan,Pakistan,STEP50305,http://documents.worldbank.org/curated/en/8892...
